In [1]:
import pandas as pd
import os
import numpy as np
import json
from collections import Counter
from collections import OrderedDict
import ast

In [2]:
# Load the data; for now we read in from the intermediate csv file
# technology = '1222'
# func_name = 'xy_canon'

In [3]:
os.getcwd()

'c:\\Users\\yihuaqia\\work\\markdraw_stat\\source_code\\dev_test\\catalog_verification'

In [4]:
def load_data(path):
    """ read in excel sheets
        grab all .xlsx files in the dir specified by path """
    files_under_path = os.listdir(path)
    fnames = [f[:-5] for f in files_under_path if f.endswith('.xlsx')]

    all_files = {}
    for i in range(len(fnames)):
        file = pd.read_excel(f'{path}/{fnames[i]}.xlsx', sheet_name=None, engine='openpyxl')
        all_files[fnames[i]] = file

    return fnames, all_files

In [5]:
path = '../../input_sheets/1227'
fnames, all_files = load_data(path)

In [6]:
all_files['canon_1227_d3t4_jm2_siv_2x'].keys()

dict_keys(['canon', 'all_functions', 'all_marks', 'parents', 'review_parent', 'dummy_beard', 'dummy_filler', 'fdr_dummy', 'kga', 'diag_dummy', 'zonal_bkg', 'xy4', 'pound', 'hatch', 'swirl_xy4', 'swirl_cross', 'swirl_hatch', 'marklist', 'delivery'])

In [7]:
import cad_functions as mfunc_table

In [8]:
mfunc_dict = mfunc_table.MARK_DRAW_FOO
reverse_mfunc_dict = {v['name']: k for k, v in mfunc_dict.items()}

In [9]:
def get_cell_info(file, test_write):
    
    for k in file.keys():
        cell_names = []
        df = pd.DataFrame(file[k])
        # check if the StartLayoutAssembler is in any place in the df
        if 'StartLayoutAssembler' not in df.values and 'zonal_background' not in df.values:
            print(f'{k} does not have StartLayoutAssembler or zonal_background')
            continue

        for i in range(df.shape[0]):
            if df.iloc[i,1] == 'StartLayoutAssembler' or df.iloc[i,1] == 'zonal_background':
                cell_names.append(df.iloc[i]['CellName.string'])
        test_write.write(f'{k}\n')
        test_write.write(f'# of unique cell names: {len(set(cell_names))}\n')
        test_write.write(f'# of cell names: {len(cell_names)}\n')
        if len(set(cell_names)) < len(cell_names):
            repated_cells = [item for item, count in Counter(cell_names).items() if count > 1]
            test_write.write(f'repeated cells: {repated_cells}\n')
        test_write.write('\n')
       

In [10]:
# with open('./tmp_write.txt', 'w+') as test_write:
#     get_cell_info(all_files['canon_1227_d3t4_jm2_siv_2x'],test_write)

In [11]:
# def get_func_seq2(file, test_write=None):
#     func_seq = []
#     for k in file.keys():
#     # for k in ['all_marks']:
#         cell_names = []
#         func_seq_lst = []
#         df = pd.DataFrame(file[k])
#         if ('StartLayoutAssembler' not in df.values) and ('zonal_background' not in df.values):
#             continue
#         # print(k, df.shape)
#         for i in range(df.shape[0]):
#             # print(f'{i}: {df.iloc[i,1]}')
#             if df.iloc[i,1] == 'StartLayoutAssembler' or df.iloc[i,1] == 'zonal_background':
#                 c_name = df.iloc[i]['CellName.string']
#                 if c_name not in cell_names:
#                     cell_names.append(c_name)
            
#             m_func = reverse_mfunc_dict.get(df.iloc[i,1], None)  # m0 ~ m13
#             if not m_func:
#                 continue

#             if m_func in ['m0','m13']:
#                 func_seq_lst.append((m_func, df.iloc[i,3]))
#             else:
#                 func_seq_lst.append((m_func, ''))
            
#         if test_write:
#             test_write.write(f'{k}\n')
#             test_write.write(f'func: {func_seq_lst}\n')
#             test_write.write('\n')
        
#         func_seq.append(func_seq_lst)
#     return func_seq

In [12]:
# with open('./tmp_write.txt', 'w+') as test_write:
#     func_seq = get_func_seq2(all_files['canon_1227_d3t4_jm2_siv_2x'],test_write)

In [13]:
def get_func_seq3(one_sheet):
    cell_names = []
    func_seq_lst = []
    df = pd.DataFrame(one_sheet)
    if ('StartLayoutAssembler' not in df.values) and ('zonal_background' not in df.values):
        return func_seq_lst
        # print(k, df.shape)
    else:
        for i in range(df.shape[0]):
            # print(f'{i}: {df.iloc[i,1]}')
            if df.iloc[i,1] == 'StartLayoutAssembler' or df.iloc[i,1] == 'zonal_background':
                c_name = df.iloc[i]['CellName.string']
                if c_name not in cell_names:
                    cell_names.append(c_name)
            
            m_func = reverse_mfunc_dict.get(df.iloc[i,1], None)  # m0 ~ m13
            if not m_func:
                continue

            if m_func in ['m0','m13']:
                func_seq_lst.append((m_func, df.iloc[i,3]))
            else:
                func_seq_lst.append((m_func, ''))
    return func_seq_lst

In [14]:
def seq_analyse2(lst):
    """ For sequnce in a given sheet, analyse the sequence of functions """
    seq = OrderedDict()  # dict of form {'cell_name': [mfunc1, mfunc2, ...], }
    n_func = len(lst)
    for i in range(n_func):
        cname = str(lst[i][1])
        
        mfunc = lst[i][0]
        if len(cname) > 0 and (cname not in seq):
            # new cell
            seq[cname] = [mfunc]
        else:
            # in-cell function calls
            last_cname = list(seq.keys())[-1]
            seq[last_cname].append(mfunc)
        
    return seq

In [15]:
# print(seq_analyse2(func_seq[1]))

In [16]:
# # get all the sequences of a file
# def all_seq(path):
#     fnames, all_files = load_data(path)
#     func_seq_all = []  # each element is a list of func_seq for a sheet
#     for k in all_files.keys():
#         func_seq = get_func_seq2(all_files[k])
#         func_seq_all.append(func_seq)
#     # return func_seq_all[0]
#     seq_analyse_all = []  # each file
#     for i in range(len(func_seq_all)):
#         seq_analyse_per_sheet = []  # each sheet
#         for j in range(len(func_seq_all[i])):
#             seq_analyse_per_sheet.append(seq_analyse2(func_seq_all[i][j]))
#         seq_analyse_all.append(seq_analyse_per_sheet)
#     return dict(zip(fnames, seq_analyse_all))

In [17]:
def all_seq2(path):
    fnames, all_files = load_data(path)
    func_seq_all = []  # each element is a list of func_seq for a sheet
    for file_name in all_files.keys():
        for sheet_name in all_files[file_name].keys():
            func_seq = get_func_seq3(all_files[file_name][sheet_name])
            func_seq_all.append(func_seq)


    result = {}
    # construct the result dict; {'file_name': {'sheet_name': {cell_name: [mfunc1, mfunc2, ...], }, }, }
    for file_name in fnames:
        result[file_name] = {}
        for sheet_name in all_files[file_name].keys():
            result[file_name][sheet_name] = seq_analyse2(func_seq_all.pop(0))

    return result


    

In [18]:
all_seq2(path)

{'canon_1227_d3t4_jm2_siv_2x': {'canon': OrderedDict([('nan', ['m0'])]),
  'all_functions': OrderedDict(),
  'all_marks': OrderedDict([('122700c_d3t4_siv_swirl', ['m13']),
               ('122700c_d3t4_jm2_dummy_3_1', ['m0', 'm15']),
               ('122700c_d3t4_siv022d', ['m0', 'm1', 'm18']),
               ('122700c_d3t4_siv023d', ['m0', 'm1', 'm18']),
               ('122700c_d3t4_siv024d', ['m0', 'm1', 'm18']),
               ('122700c_d3t4_siv078d', ['m0', 'm1', 'm18']),
               ('122700c_d3t4_siv173d', ['m0', 'm7', 'm18']),
               ('122700c_d3t4_siv222d', ['m0', 'm1', 'm1', 'm12', 'm18']),
               ('122700c_d3t4_siv223d', ['m0', 'm1', 'm1', 'm12', 'm18']),
               ('122700c_d3t4_siv224d', ['m0', 'm1', 'm1', 'm12', 'm18']),
               ('122700c_d3t4_siv355d',
                ['m13', 'm0', 'm16', 'm6', 'm6', 'm12', 'm10']),
               ('122700c_d3t4_siv374d',
                ['m13', 'm0', 'm16', 'm7', 'm7', 'm12', 'm10'])]),
  'parents': Ordere

In [19]:
# seq_analyse_per_file = all_seq(path)

In [20]:
# seq_analyse_per_file.keys()

In [21]:
# len(seq_analyse_per_file['canon_1227_d3t4_jm2_siv_2x'])

In [22]:
# seq_analyse_per_file['canon_1227_d3t4_jm2_siv_2x'][1].keys()

In [23]:
# seq_analyse_per_file['canon_1227_d3t4_jm2_siv_2x'][1]['122700c_d3t4_siv355d']

# Compare with current Cannon catalogue

In [24]:
from mark_catalogs import CANON_CATALOG as cc
# from mark_catalogs_old import CANON_CATALOG as cc_old

In [25]:
def get_catalog_seq(cc):
    catalog_seq = {}
    for k in cc.keys():
        catalog_seq[k] = list(cc[k]['CAD_info']['function_sequence'])
    return catalog_seq

In [26]:
catalog_seq = get_catalog_seq(cc)

In [27]:
catalog_seq

{'CAXXID': ['m0', 'm17'],
 'CAXXD1': ['m0', 'm21'],
 'CAXXD2': ['m0', 'm20'],
 'CAXXD3': ['m0', 'm15'],
 'CAXXD4': ['m0', 'm15'],
 'CAXXD5': ['m0', 'm14'],
 'CAXXD6': ['m0', 'm14'],
 'CA4X1': ['m0', 'm1', 'm14', 'm10'],
 'CA4X2': ['m0', 'm1', 'm14', 'm10'],
 'CA4X3': ['m0', 'm6', 'm14', 'm10'],
 'CA4X4': ['m0', 'm7', 'm14', 'm10'],
 'CA4X5': ['m0', 'm1', 'm14', 'm10'],
 'CA4X6': ['m0', 'm1', 'm14', 'm10'],
 'CA4X7': ['m0', 'm6', 'm14', 'm10'],
 'CA4X8': ['m0', 'm7', 'm14', 'm10'],
 'CA4X9': ['m13', 'm0', 'm1', 'm10'],
 'CA2X1': ['m0', 'm1', 'm14', 'm10'],
 'CA2X2': ['m0', 'm1', 'm14', 'm10'],
 'CA2X3': ['m0', 'm6', 'm14', 'm10'],
 'CA2X4': ['m0', 'm7', 'm14', 'm10'],
 'CA2X5': ['m0', 'm1', 'm14', 'm10'],
 'CA2X6': ['m0', 'm1', 'm14', 'm10'],
 'CA2X7': ['m0', 'm6', 'm14', 'm10'],
 'CA2X8': ['m0', 'm7', 'm14', 'm10'],
 'CA2X9': ['m0', 'm1'],
 'CA2X10': ['m0', 'm1'],
 'CA2X11': ['m0', 'm6'],
 'CA2X12': ['m0', 'm7'],
 'CA2X13': ['m0', 'm1', 'm13'],
 'CA2X14': ['m0', 'm3'],
 'CA2X15': ['m0'

In [28]:
len(catalog_seq)

35

In [29]:
# print the key if the value is found in the dict
def get_key(val, my_dict):
    for key, value in my_dict.items():
        if val == value:
            return key
    return None

In [30]:
print(get_key(['m0','m13'], catalog_seq))
print(get_key(['m0','m3'], catalog_seq))

None
CA2X14


In [31]:
# def compare(path, test_write=None):
#     """compare the sequence of functions in the input sheets with the catalog"""
#     seq_analyse_per_file = all_seq(path)
#     catalog_seq = get_catalog_seq()
#     for file in seq_analyse_per_file.keys():
#         if test_write:
#             test_write.write(f'For {file} seq not found in catalog:\n')
#         for cell in seq_analyse_per_file[file]:
#             for cell_name in cell.keys():
#                 if cell[cell_name] not in catalog_seq.values():
#                     if test_write:
#                         test_write.write(f'{cell_name}: {cell[cell_name]}\n')
#                     # print(f'{cell_name} in {file} is not in the catalog\n')
#         test_write.write('\n')
        

In [32]:
# with open('./catalog_compare.txt', 'w+') as test_write:
#     compare('../../input_sheets/1227', test_write=test_write)

In [33]:
# with open('./catalog_compare1276.txt', 'w+') as test_write:
#     compare('../../input_sheets/1276', test_write=test_write)

In [34]:
# def compare2(path, test_write=None):
#     """compare the sequence of functions in the input sheets with the catalog"""
#     seq_analyse_per_file = all_seq(path)
#     catalog_seq = get_catalog_seq()
#     for file in seq_analyse_per_file.keys():
#         if test_write:
#             test_write.write(f'For {file} seq not found in catalog:\n')

#         for cell in seq_analyse_per_file[file]:
#             for cell_name in cell.keys():
#                 seq_clean = [m for m in cell[cell_name] if m != 'm18']  # remove m18
#                 if seq_clean not in catalog_seq.values():
#                     if test_write:
#                         test_write.write(f'{cell_name}: {cell[cell_name]}\n')
#                     # print(f'{cell_name} in {file} is not in the catalog\n')
#         test_write.write('\n')
        

In [35]:
def compare3(path, test_write=None):
    """compare the sequence of functions in the input sheets with the catalog"""
    seq_analyse = all_seq2(path)
    catalog_seq = get_catalog_seq(cc)
    for file in seq_analyse.keys():
        if test_write:
            test_write.write(f'For {file} seq not found in catalog:\n')

        for sheet in seq_analyse[file].keys():
            for cell in seq_analyse[file][sheet]:
                seq = seq_analyse[file][sheet][cell]
                seq_clean = [m for m in seq if m != 'm18']
                if seq_clean not in catalog_seq.values():
                    print(f'{file} {sheet} {cell}: {seq_clean}')
                    # print(f'{cell_name} in {file} is not in the catalog\n')

        

In [ ]:
# compare3('../../input_sheets/1222')
    

canon_1222_d4t1t5_S22Y_GMB_GV1_2x canon nan: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_f_01: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_f_02: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_f_03: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_f_04: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_s_05: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_f_06: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_f_07: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_f_08: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_s_09: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x parents 122200c_d4t1t5_s22y_frm_1x1_canon_s_10: ['m0']
canon_1222_d4t1t5_S22Y_GMB_GV1_2x dummy_and_beard 122200c_

In [37]:
# with open('./catalog_compare_1222_c.txt', 'w+') as test_write:
#     compare2('../../input_sheets/1222', test_write=test_write)

In [38]:
# with open('./catalog_compare_1227_c.txt', 'w+') as test_write:
#     compare2('../../input_sheets/1227', test_write=test_write)

In [39]:
def match_cats(catalog_seq, seq):
    # given the sequence of functions in the input sheets, 
    # find the matching sequence in the catalog_seq and retrun the key
    r = []
    reduced_seq = [m for m in seq if m != 'm18']
    for k in catalog_seq.keys():
        if catalog_seq[k] == reduced_seq:
            r.append(k)
    return r

In [137]:
def seq_cata(in_path, out_fname=None):
    """
        Compare the sequence of functions in the input sheets with the catalog.
        Record the function sequences and catalog matches in a csv file.
    """
    # get all the func sequences in the input sheets
    # of form {cell_name: [mfunc1, mfunc2, ...], }
    seq_analyse = all_seq2(in_path)
    catalog_seq = get_catalog_seq(cc)  # catalog to compare with
    df = pd.DataFrame(columns=['file', 'sheet', 'cell', 'seq', 'catalog'])
    for file in seq_analyse.keys():
        for sheet in seq_analyse[file].keys():
            for cell in seq_analyse[file][sheet]:
                seq = seq_analyse[file][sheet][cell]
                seq_clean = [m for m in seq if m != 'm18']
                # print(f'{file} {sheet} {cell}: {seq_clean}')
    #         for seq in cell:
                newline = {'file': file, 'sheet': sheet, 'cell': cell, 'seq': str(seq), 'catalog': str(match_cats(catalog_seq, seq_clean))}
                df.loc[len(df)] = newline
    #             # print(f"{file}, {cell_name}: {seq}")
    if out_fname:
        df.to_csv(out_fname, index=False)
    return df
    

In [41]:
seq_cata('../../input_sheets/1227', 'tmp.csv')

In [42]:
# cata_refine('tmp.csv')

## input_to_df

In [148]:
def get_cell_info2(file):
    """ get the cell names from the StartLayoutAssembler or zonal_background """
    # df_list = []
    od_df = OrderedDict()
    name_list = []
    for sname in file.keys():
        cell_names = []
        new_cell_idx = []
        df = pd.DataFrame(file[sname])
        # check if the StartLayoutAssembler is in any place in the df
        if 'StartLayoutAssembler' not in df.values and 'zonal_background' not in df.values:
            continue
        
        # get cell names; i is the row index
        for i in range(df.shape[0]):
            # print(f'{i}: {df.iloc[i,1]}')
            if df.iloc[i,1] == 'StartLayoutAssembler' or df.iloc[i,1] == 'zonal_background':
                c_name = df.iloc[i]['CellName.string']

                if c_name not in cell_names:
                    # cell_names.append((c_name,i))
                    cell_names.append(c_name)
                    new_cell_idx.append(i)

        # print(f'{fname}: {cell_names}')
        # slice the cells based on cellnames
        df_cells = {} 
        for i in range(len(cell_names)-1):
            df_cells[cell_names[i]] = df.iloc[new_cell_idx[i]:new_cell_idx[i+1],:].dropna(how='all')
        #     df_cells[cell_names[i][0]] = df.iloc[cell_names[i][1]:cell_names[i+1][1],:].dropna(how='all')
        # df_cells[cell_names[-1][0]] = df.iloc[cell_names[-1][1]:,:].dropna(how='all')
        df_cells[cell_names[-1]] = df.iloc[new_cell_idx[-1]:,:].dropna(how='all')
        
        # df_list.append(df_cells)
        od_df[sname] = df_cells
        name_list.append(cell_names)
    return od_df, name_list


def get_df_info(technology,file_name, sheet_name, cell_name):
    """ get the cell information from the file """
    in_file_path = f'../../input_sheets/{technology}'
    fnames, all_files = load_data(in_file_path)
    od_df, name_lst = get_cell_info2(all_files[file_name])
    return od_df[sheet_name][cell_name]

In [149]:
get_df_info(1227,'canon_1227_d3t4_jm2_siv_2x', 'all_marks', '122700c_d3t4_jm2_dummy_3_1')

,EXECUTE,SampleStartLayoutAssembler,Library.string,CellName.string,OpenCellViewMode.string,CellSizeX.float,CellSizeY.float,CenterX.float,CenterY.float,Category.string
16,y,StartLayoutAssembler,canon27d3t4lib1_p27_lay,122700c_d3t4_jm2_dummy_3_1,w,59.74,61.72,0,0,NaN
17,y,diagonal_gratings,cv,JM2_mask.frameDrawing,135,2.004,3.006,2.2,0,NaN


Lift the degeneracy of the catalog

In [158]:
def degen_lift(df_orig, degen_cata):
    degen_cata = ast.literal_eval(degen_cata)
    if degen_cata == ['CAXXD3', 'CAXXD4']:
        # print(df_orig)
        row_label = 'diagonal_gratings'
        col_label = 'OpenCellViewMode.string'
        # get the row containing the diagonal_gratings
        df_local = df_orig[df_orig['SampleStartLayoutAssembler'] == row_label]
        if df_local[col_label].values[0] == 45:
            degen_cata = ['CAXXD4']
        else:
            degen_cata = ['CAXXD3']

    if degen_cata == ['CA2X9', 'CA2X10']:
        row_label = 'xy_canon'
        col_label1 = 'CellSizeX.float'
        col_label2 = 'CellSizeY.float'
        df_local = df_orig[df_orig['SampleStartLayoutAssembler'] == row_label]
        if df_local[col_label1].values[0] < df_local[col_label2].values[0]:
            degen_cata = ['CA2X9']
        else:
            degen_cata = ['CA2X10']

    if degen_cata == ['CA2X14', 'CA2X15']:
        row_label = 'fa_canon'
        col_label = 'CenterY.float'
        df_local = df_orig[df_orig['SampleStartLayoutAssembler'] == row_label]
        if df_local[col_label].values[0] == 'v':
            degen_cata = ['CA2X14']
        elif df_local[col_label].values[0] == 'h':
            degen_cata = ['CA2X15']

    if degen_cata == ['CAXXD5', 'CAXXD6']:
        row_label = 'create_gratings'
        col_label = 'CenterX.float'
        df_local = df_orig[df_orig['SampleStartLayoutAssembler'] == row_label]
        if df_local[col_label].values[0] == 'h':
            degen_cata = ['CAXXD5']
        elif df_local[col_label].values[0] == 'v':
            degen_cata = ['CAXXD6']

    return degen_cata

In [151]:
def cata_refine(seq_cata_file=None, df_in=None, technology=None):
    if seq_cata_file:
        with open(seq_cata_file, 'r') as f:
            df = pd.read_csv(f)
    else:
        df = df_in
    
    for idx, row in df.iterrows():
        catalogs = ast.literal_eval(row['catalog'])  # convert string to list
        if len(catalogs) > 1:
            # print(f'{row["file"]}, {row["cell"]}: {row["seq"]}, {row["catalog"]}')
            # inplace update the catalog
            df.at[idx, 'catalog'] = degen_lift(get_df_info(technology, row['file'], row['sheet'], row['cell']), row["catalog"])

    return df


In [152]:
# df_degen_lifted = cata_refine(seq_cata_file='tmp.csv', technology=1227)
# df_degen_lifted.to_csv('tmp_refined.csv', index=False)

In [153]:
# df_test = pd.read_csv('tmp.csv')
# df_test.head()

In [154]:
# df_test.iloc[2]['catalog']

In [159]:
def main():
    if not os.path.exists('refined_catalogs'):
        os.mkdir('refined_catalogs')
    technologies = ['1222', '1227', '1276', '1278', '1280', '1282']
    for t in technologies:
        path = f'../../input_sheets/{t}'
        df = seq_cata(path)
        df = cata_refine(df_in=df, technology=t)
        df.to_csv(f'refined_catalogs/{t}_refined.csv', index=False)

In [160]:
main()